In [1]:
import numpy as np
from faker import Faker
from freamon import random_subset, load_data, integrate_data, compute_feature_and_label_data, define_model
from arguseyes import ArgusEyes
import logging
from arguseyes.refinements import InputUsage, FairnessMetrics

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)

In [2]:
def execute_with_arguseyes(seed):
    
    cmd_args=[str(seed)]

    eq_opp = 0
    rating_usage = None
    product_usage = None

    eyes = ArgusEyes('freamon-testing', './mlruns')
    with eyes.classification_pipeline_from_py_file('freamon.py', cmd_args=cmd_args) as pipeline:

        tagged_inputs = pipeline.compute(InputUsage())
        priv_tn, priv_fp, priv_fn, priv_tp, dis_tn, dis_fp, dis_fn, dis_tp = \
            pipeline.compute(FairnessMetrics('third_party', 'Y'))

        eq_opp = (float(dis_fn) / (dis_fn + dis_tp)) - (float(priv_fn) / (priv_fn + priv_tp))
        #print('Equal opportunity', eq_opp)

        for tagged_input in tagged_inputs:
            column_names = list(tagged_input.data.columns)

            if 'star_rating' in column_names:
                rating_usage = np.array(tagged_input.data['__arguseyes__is_used'])
                #print('# ratings used', np.sum(rating_usage))

            if 'product_title' in column_names:
                product_usage = np.array(tagged_input.data['__arguseyes__is_used'])
                #print('# products used', np.sum(product_usage))  


    return rating_usage, product_usage, eq_opp            

In [3]:
def execute_manually(seed):
    
    fake = Faker()
    fake.seed_instance(seed)
    np.random.seed(seed)
    
    numerical_columns = random_subset(['total_votes', 'star_rating'])
    categorical_columns = random_subset(['customer_id', 'product_id', 'vine', 'category'])
    final_columns = list(set(numerical_columns + categorical_columns + \
      ['text', 'is_helpful', 'review_date'] + ['third_party', 'product_id', 'rating_id']))

    reviews, ratings, products, categories = load_data()
    
    ratings['rating_id'] = range(len(ratings))
    
    integrated_data = integrate_data(reviews, ratings, products, categories, fake)
    train_data, train_labels, test_data, test_labels = \
        compute_feature_and_label_data(integrated_data, final_columns, fake)
    
    sklearn_model = define_model(numerical_columns, categorical_columns)
    
    model = sklearn_model.fit(train_data, train_labels)

    #print('Train accuracy', model.score(train_data, train_labels))
    #print('Test accuracy', model.score(test_data, test_labels))
    
    # Group fairness computation
    from sklearn.metrics import confusion_matrix

    eval_data = test_data.copy(deep=True)

    eval_data['true'] = test_labels
    eval_data['pred'] = model.predict(test_data)
    
    priv = eval_data[eval_data.third_party=='Y']
    dis = eval_data[eval_data.third_party=='N']

    priv_tn, priv_fp, priv_fn, priv_tp = confusion_matrix(priv['true'], priv['pred']).ravel()
    dis_tn, dis_fp, dis_fn, dis_tp = confusion_matrix(dis['true'], dis['pred']).ravel()

    eq_opp = (float(dis_fn) / (dis_fn + dis_tp)) - (float(priv_fn) / (priv_fn + priv_tp))

    #print('Equal opportunity', eq_opp)
    
    # Data usage
    train_rating_ids = list(set(train_data.rating_id))
    train_product_ids = list(set(train_data.product_id))

    rating_usage = np.array(ratings.rating_id.isin(train_rating_ids).values)
    product_usage = np.array(products.product_id.isin(train_product_ids).values)
    
    #print('# ratings used', np.sum(rating_usage))
    #print('# products used', np.sum(product_usage))
    
    return rating_usage, product_usage, eq_opp

In [4]:
seeds = [1234, 830848, 987496, 429214, 178132, 590519, 725202, 476388, 133576, 515078, 
         720668, 449660, 168556, 74155, 164680, 142209, 181459, 994015, 172466, 529630]


with open("random-testing-log.txt", 'w') as logfile:
    for seed in seeds:
        manual_rating_usage, manual_product_usage, manual_eq_opp = execute_manually(seed)
        arg_rating_usage, arg_product_usage, arg_eq_opp = execute_with_arguseyes(seed)

        print(f'MANUAL, seed={seed}, # ratings used {np.sum(manual_rating_usage)}, ' +
              f'# products used {np.sum(manual_product_usage)}, eq_opp {manual_eq_opp}')

        logfile.write(f'MANUAL, seed={seed}, # ratings used {np.sum(manual_rating_usage)}, ' +
              f'# products used {np.sum(manual_product_usage)}, eq_opp {manual_eq_opp}\n')

        print(f'AUTO, seed={seed}, # ratings used {np.sum(arg_rating_usage)}, ' +
              f'# products used {np.sum(arg_product_usage)}, eq_opp {arg_eq_opp}')

        logfile.write(f'AUTO, seed={seed}, # ratings used {np.sum(arg_rating_usage)}, ' +
              f'# products used {np.sum(arg_product_usage)}, eq_opp {arg_eq_opp}\n')

        ratings_match = np.array_equal(manual_rating_usage, arg_rating_usage)
        products_match = np.array_equal(manual_product_usage, arg_product_usage)
        eq_opps_match = manual_eq_opp == arg_eq_opp

        print(f'COMPARISON: {ratings_match}, {products_match}, {eq_opps_match}')       
        logfile.write(f'COMPARISON: {ratings_match}, {products_match}, {eq_opps_match}\n')

        print('---------------------------------------\n')              


INFO:Patching sys.argv with ['eyes', '1234']
INFO:Created run 08063cbf32cb46efb37909b8bf1f654c for this invocation
INFO:Executing instrumented user pipeline with mlinspect
INFO:Redirecting the pipeline's stdout to arguseyes-pipeline-output.txt
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/mlinspect/monkeypatching/_monkey_patching_utils.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result = original_func(*args, **kwargs)
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

Equal opportunity 0.0337270493512396
# products used 3088
# ratings used 23440
MANUAL, seed=1234, # ratings used 23440, # products used 3088, eq_opp 0.0337270493512396
AUTO, seed=1234, # ratings used 23440, # products used 3088, eq_opp 0.0337270493512396
COMPARISON: True, True, True
---------------------------------------



INFO:Patching sys.argv with ['eyes', '830848']
INFO:Created run 8827b79886df497b90f6e3adc71ade7c for this invocation
INFO:Executing instrumented user pipeline with mlinspect
INFO:Redirecting the pipeline's stdout to arguseyes-pipeline-output.txt
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/mlinspect/monkeypatching/_monkey_patching_utils.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result = original_func(*args, **kwargs)
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

Equal opportunity 0.016491953858055915
# products used 4328
# ratings used 38469
MANUAL, seed=830848, # ratings used 38469, # products used 4328, eq_opp 0.016491953858055915
AUTO, seed=830848, # ratings used 38469, # products used 4328, eq_opp 0.016491953858055915
COMPARISON: True, True, True
---------------------------------------



INFO:Patching sys.argv with ['eyes', '987496']
INFO:Created run 576f8f1eed094c0cb9eb1a28493b949a for this invocation
INFO:Executing instrumented user pipeline with mlinspect
INFO:Redirecting the pipeline's stdout to arguseyes-pipeline-output.txt
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/mlinspect/monkeypatching/_monkey_patching_utils.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result = original_func(*args, **kwargs)
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

...

Equal opportunity 0.02723642435095551
# products used 3777
# ratings used 30876
MANUAL, seed=994015, # ratings used 30876, # products used 3777, eq_opp 0.02723642435095551
AUTO, seed=994015, # ratings used 30876, # products used 3777, eq_opp 0.02723642435095551
COMPARISON: True, True, True
---------------------------------------



INFO:Patching sys.argv with ['eyes', '172466']
INFO:Created run a52a230846374a62b627230ade32712e for this invocation
INFO:Executing instrumented user pipeline with mlinspect
INFO:Redirecting the pipeline's stdout to arguseyes-pipeline-output.txt
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/mlinspect/monkeypatching/_monkey_patching_utils.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result = original_func(*args, **kwargs)
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

Equal opportunity 0.03685799224782671
# products used 4858
# ratings used 48801
MANUAL, seed=172466, # ratings used 48801, # products used 4858, eq_opp 0.03685799224782671
AUTO, seed=172466, # ratings used 48801, # products used 4858, eq_opp 0.03685799224782671
COMPARISON: True, True, True
---------------------------------------



INFO:Patching sys.argv with ['eyes', '529630']
INFO:Created run 1da8bd3779644abd96bb5578f55cbec6 for this invocation
INFO:Executing instrumented user pipeline with mlinspect
INFO:Redirecting the pipeline's stdout to arguseyes-pipeline-output.txt
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/mlinspect/monkeypatching/_monkey_patching_utils.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result = original_func(*args, **kwargs)
/Users/ssc/projects/arguseyes/venv/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

Equal opportunity 0.01282589660341013
# products used 3688
# ratings used 29738
MANUAL, seed=529630, # ratings used 29738, # products used 3688, eq_opp 0.01282589660341013
AUTO, seed=529630, # ratings used 29738, # products used 3688, eq_opp 0.01282589660341013
COMPARISON: True, True, True
---------------------------------------

